In [ ]:
# 필요한 라이브러리 설치
!pip install openai==0.28
!pip install pandas

import os
import pandas as pd
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import openai

# OpenAI API 키 설정
openai.api_key = 'sk-'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def load_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

def evaluate_esg(sentence):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 ESG 평가 전문가입니다. 먼저 ESG에 대한 설명을 제공해드릴테니 평가의 기준으로 활용하시기 바랍니다. ESG는 Environmental(환경), Social(사회), Governance(거버넌스)의 세 가지 측면으로 구성됩니다. 환경 측면은 기업이 환경 친화적으로 운영하고자 하는 의지를 나타냅니다. 이는 친환경 제품 제조, 에너지 효율성 향상 등을 포함합니다. 사회적 측면은 기업이 사회적 책임을 다하는 것을 의미하며, 노동자 권리 보호, 다양성과 포용성 강화 등이 포함됩니다. 거버넌스 측면은 기업 내부의 투명성과 윤리성을 강화하여 이해관계자들의 신뢰를 유지하는 것을 중요시합니다. 이는 감사와 감사위원회의 설정, 보상 정책의 공정성 등을 포함합니다. 이러한 ESG 요소들은 기업의 장기적인 지속 가능성과 가치 창출에 영향을 미치며, 적절한 관리를 통해 사회적으로 책임 있는 기업 경영을 추구합니다. 이제부터, 주어지는 문장을 ESG 긍정인 경우 1, ESG 중립인 경우 0, ESG 부정인 경우 -1로 분류하세요. 결과는 무조건 1,0,-1 셋 중 하나여야만 합니다.  문장으로 출력하지 마십시오. 예를 들어, 환경에 대해 긍정적인 문장이라면 ESG 긍정이므로 반환값은 '1'입니다. 이제 평가를 시작하십시오. "},
            {"role": "user", "content": sentence}
        ]
    )
    result = response['choices'][0]['message']['content']
    if "1" in result:
        return 1
    elif "0" in result:
        return 0
    elif "-1" in result:
        return -1
    return result  # 기본값

def calculate_document_esg_score(sentences):
    """문서의 ESG 친화도 총점을 계산합니다."""
    score = 0
    for sentence in sentences:
        score += evaluate_esg(sentence)  # 점수를 더합니다.
    return score

def process_documents(directory_path):
    # 결과를 저장할 빈 리스트
    documents_scores = []

    # 지정된 디렉토리 내의 모든 파일을 순회
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  # 텍스트 파일만 처리
            file_path = os.path.join(directory_path, filename)
            text = load_text_from_file(file_path)
            sentences = split_into_sentences(text)
            document_score = calculate_document_esg_score(sentences)
            documents_scores.append({'Document': filename, 'ESG Score': document_score})

    # 결과 데이터프레임 생성
    df_scores = pd.DataFrame(documents_scores)
    return df_scores


In [ ]:

# 문서가 저장된 디렉토리 경로
directory_path = "/content/drive/MyDrive/openaitest/"
df_scores = process_documents(directory_path)

# 결과 출력
print(df_scores)

# 선택적: 결과 데이터프레임을 CSV 파일로 저장
df_scores.to_csv('/content/drive/MyDrive/esg_document_scores.csv', index=False)


          Document  ESG Score
0   1_report_2.txt         12
1   1_report_1.txt         12
2   2_report_1.txt          9
3   2_report_2.txt         11
4  2_article_2.txt         12
5  1_article_1.txt         10
6  1_article_2.txt          9
7  2_article_1.txt          9


In [ ]:
print(df_scores)


          Document  ESG Score
0   1_report_2.txt         12
1   1_report_1.txt         12
2   2_report_1.txt          9
3   2_report_2.txt         11
4  2_article_2.txt         12
5  1_article_1.txt         10
6  1_article_2.txt          9
7  2_article_1.txt          9


In [ ]:
import pandas as pd

# 'Document' 열을 분석하여 새로운 정보를 추출
df_scores['기업번호'] = df_scores['Document'].apply(lambda x: x.split('_')[0])
df_scores['문서유형'] = df_scores['Document'].apply(lambda x: x.split('_')[1])
df_scores['발행정보'] = df_scores['Document'].apply(lambda x: x.split('_')[2])

# 같은 기업의 보고서와 기사 별로 그룹화하여 점수의 평균과 합산 계산
result = df_scores.groupby(['기업번호', '문서유형'])['ESG Score'].agg(['mean', 'sum']).reset_index()

# 결과 출력
print(result)


  기업번호     문서유형  mean  sum
0    1  article   9.5   19
1    1   report  12.0   24
2    2  article  10.5   21
3    2   report  10.0   20
